In [5]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StringType

def get_products_with_categories(products_df: DataFrame,
                               categories_df: DataFrame,
                               product_category_df: DataFrame) -> DataFrame:

    # Проверяем наличие необходимых колонок
    required_product_cols = ['product_id', 'product_name']
    required_category_cols = ['category_id', 'category_name']
    required_relation_cols = ['product_id', 'category_id']

    for df, cols, name in zip(
        [products_df, categories_df, product_category_df],
        [required_product_cols, required_category_cols, required_relation_cols],
        ['products_df', 'categories_df', 'product_category_df']
    ):
        if not all(col in df.columns for col in cols):
            raise ValueError(f"В {name} отсутствуют необходимые колонки: {cols}")

    # Получаем все пары "Продукт - Категория"
    product_category_pairs = (
        product_category_df
        .join(products_df, "product_id", "inner")
        .join(categories_df, "category_id", "inner")
        .select("product_name", "category_name")
    )

    # Получаем продукты без категорий
    products_without_categories = (
        products_df
        .join(product_category_df, "product_id", "left_anti")
        .select(
            col("product_name"),
            lit(None).cast(StringType()).alias("category_name")
        )
    )

    # Объединяем результаты
    result_df = product_category_pairs.union(products_without_categories)

    return result_df.orderBy("product_name", "category_name")

In [6]:
import unittest
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col


class TestProductsWithCategories(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        cls.spark = SparkSession.builder \
            .appName("TestProductsWithCategories") \
            .master("local[2]") \
            .config("spark.sql.adaptive.enabled", "false") \
            .getOrCreate()

    @classmethod
    def tearDownClass(cls):
        cls.spark.stop()

    def setUp(self):
        # Создаем тестовые данные
        self.products_data = [
            ("p1", "Product A"),
            ("p2", "Product B"),
            ("p3", "Product C"),
            ("p4", "Product D"),
            ("p5", "Product E")
        ]

        self.categories_data = [
            ("c1", "Category 1"),
            ("c2", "Category 2"),
            ("c3", "Category 3")
        ]

        self.relations_data = [
            ("p1", "c1"),
            ("p1", "c2"),
            ("p2", "c1"),
            ("p3", "c3"),
            # p4 нет категорий
            # p5 нет категорий
        ]

        self.products_df = self.spark.createDataFrame(
            self.products_data,
            ["product_id", "product_name"]
        )

        self.categories_df = self.spark.createDataFrame(
            self.categories_data,
            ["category_id", "category_name"]
        )

        self.relations_df = self.spark.createDataFrame(
            self.relations_data,
            ["product_id", "category_id"]
        )

    def assertDataFramesEqual(self, df1, df2):
        """Кастомный assert для сравнения датафреймов"""
        # Сравниваем схемы
        self.assertEqual(df1.schema, df2.schema)

        # Собираем данные и сравниваем
        data1 = sorted(df1.collect(), key=lambda x: (x['product_name'] or '', x['category_name'] or ''))
        data2 = sorted(df2.collect(), key=lambda x: (x['product_name'] or '', x['category_name'] or ''))

        self.assertEqual(len(data1), len(data2))

        for row1, row2 in zip(data1, data2):
            self.assertEqual(row1['product_name'], row2['product_name'])
            self.assertEqual(row1['category_name'], row2['category_name'])

    def test_basic_functionality(self):
        """Тест базовой функциональности"""
        result_df = get_products_with_categories(
            self.products_df,
            self.categories_df,
            self.relations_df
        )

        expected_data = [
            ("Product A", "Category 1"),
            ("Product A", "Category 2"),
            ("Product B", "Category 1"),
            ("Product C", "Category 3"),
            ("Product D", None),
            ("Product E", None)
        ]

        expected_df = self.spark.createDataFrame(
            expected_data,
            ["product_name", "category_name"]
        ).orderBy("product_name", "category_name")

        result_sorted = result_df.orderBy("product_name", "category_name")
        self.assertDataFramesEqual(result_sorted, expected_df)

    def test_empty_relations(self):
        """Тест когда нет связей между продуктами и категориями"""
        empty_relations_df = self.spark.createDataFrame(
            [],
            StructType([
                StructField("product_id", StringType()),
                StructField("category_id", StringType())
            ])
        )

        result_df = get_products_with_categories(
            self.products_df,
            self.categories_df,
            empty_relations_df
        )

        # Все продукты должны быть без категорий
        self.assertEqual(result_df.count(), 5)
        self.assertEqual(
            result_df.filter(col("category_name").isNull()).count(),
            5
        )

    def test_empty_products(self):
        """Тест когда нет продуктов"""
        empty_products_df = self.spark.createDataFrame(
            [],
            StructType([
                StructField("product_id", StringType()),
                StructField("product_name", StringType())
            ])
        )

        result_df = get_products_with_categories(
            empty_products_df,
            self.categories_df,
            self.relations_df
        )

        # Результат должен быть пустым
        self.assertEqual(result_df.count(), 0)

    def test_empty_categories(self):
        """Тест когда нет категорий"""
        empty_categories_df = self.spark.createDataFrame(
            [],
            StructType([
                StructField("category_id", StringType()),
                StructField("category_name", StringType())
            ])
        )

        result_df = get_products_with_categories(
            self.products_df,
            empty_categories_df,
            self.relations_df
        )

        # Все продукты должны быть без категорий
        self.assertEqual(result_df.count(), 5)
        self.assertEqual(
            result_df.filter(col("category_name").isNull()).count(),
            5
        )

    def test_missing_columns(self):
        """Тест проверки отсутствующих колонок"""
        products_wrong_cols = self.spark.createDataFrame(
            [("p1", "Product A")],
            ["id", "name"] # Неправильные названия колонок
        )

        with self.assertRaises(ValueError):
            get_products_with_categories(
                products_wrong_cols,
                self.categories_df,
                self.relations_df
            )

    def test_duplicate_relations(self):
        """Тест с дублирующимися связями"""
        duplicate_relations_data = [
            ("p1", "c1"),
            ("p1", "c1"), # Дубликат
            ("p1", "c2"),
            ("p2", "c1")
        ]

        duplicate_relations_df = self.spark.createDataFrame(
            duplicate_relations_data,
            ["product_id", "category_id"]
        )

        result_df = get_products_with_categories(
            self.products_df,
            self.categories_df,
            duplicate_relations_df
        )

        # Дубликаты должны быть удалены
        product_a_categories = result_df.filter(col("product_name") == "Product A")
        self.assertEqual(product_a_categories.count(), 2) # Только 2 уникальные категории

    def test_products_with_multiple_categories(self):
        """Тест продуктов с несколькими категориями"""
        result_df = get_products_with_categories(
            self.products_df,
            self.categories_df,
            self.relations_df
        )

        # Product A имеет 2 категории
        product_a_categories = result_df.filter(col("product_name") == "Product A")
        self.assertEqual(product_a_categories.count(), 2)

        # Проверяем конкретные категории
        categories = [row.category_name for row in product_a_categories.collect()]
        self.assertIn("Category 1", categories)
        self.assertIn("Category 2", categories)

    def test_products_without_categories(self):
        """Тест продуктов без категорий"""
        result_df = get_products_with_categories(
            self.products_df,
            self.categories_df,
            self.relations_df
        )

        # Продукты без категорий
        products_without_cats = result_df.filter(col("category_name").isNull())
        self.assertEqual(products_without_cats.count(), 2)

        product_names = [row.product_name for row in products_without_cats.collect()]
        self.assertIn("Product D", product_names)
        self.assertIn("Product E", product_names)


if __name__ == "__main__":
    unittest.main()


E
ERROR: /root/ (unittest.loader._FailedTest./root/)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: 1

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
def main():
    """Пример использования метода"""
    spark = SparkSession.builder \
        .appName("ProductsWithCategoriesExample") \
        .master("local[2]") \
        .getOrCreate()

    # Создаем пример данных
    products_data = [
        ("1", "Laptop"),
        ("2", "Smartphone"),
        ("3", "Tablet"),
        ("4", "Headphones"),
        ("5", "Camera")
    ]

    categories_data = [
        ("1", "Electronics"),
        ("2", "Computers"),
        ("3", "Mobile Devices"),
        ("4", "Audio")
    ]

    relations_data = [
        ("1", "1"), # Laptop -> Electronics
        ("1", "2"), # Laptop -> Computers
        ("2", "1"), # Smartphone -> Electronics
        ("2", "3"), # Smartphone -> Mobile Devices
        ("4", "1"), # Headphones -> Electronics
        ("4", "4"), # Headphones -> Audio
        # Tablet и Camera без категорий
    ]

    products_df = spark.createDataFrame(products_data, ["product_id", "product_name"])
    categories_df = spark.createDataFrame(categories_data, ["category_id", "category_name"])
    relations_df = spark.createDataFrame(relations_data, ["product_id", "category_id"])

    # Используем наш метод
    result_df = get_products_with_categories(products_df, categories_df, relations_df)

    print("Результат:")
    result_df.show()

    # Анализ результатов
    print("\nСтатистика:")
    print(f"Всего записей: {result_df.count()}")
    print(f"Продуктов с категориями: {result_df.filter(col('category_name').isNotNull()).count()}")
    print(f"Продуктов без категорий: {result_df.filter(col('category_name').isNull()).count()}")

    spark.stop()

if __name__ == "__main__":
    main()


Результат:
+------------+--------------+
|product_name| category_name|
+------------+--------------+
|      Camera|          NULL|
|  Headphones|         Audio|
|  Headphones|   Electronics|
|      Laptop|     Computers|
|      Laptop|   Electronics|
|  Smartphone|   Electronics|
|  Smartphone|Mobile Devices|
|      Tablet|          NULL|
+------------+--------------+


Статистика:
Всего записей: 8
Продуктов с категориями: 6
Продуктов без категорий: 2


In [9]:
# quick_test.py
from pyspark.sql import SparkSession

def quick_test():
    spark = SparkSession.builder \
        .appName("QuickTest") \
        .master("local[1]") \
        .getOrCreate()

    # Простые тестовые данные
    products = spark.createDataFrame([
        ("1", "Laptop"), ("2", "Phone"), ("3", "Tablet")
    ], ["product_id", "product_name"])

    categories = spark.createDataFrame([
        ("1", "Electronics"), ("2", "Computers")
    ], ["category_id", "category_name"])

    relations = spark.createDataFrame([
        ("1", "1"), ("1", "2"), ("2", "1")
    ], ["product_id", "category_id"])

    result = get_products_with_categories(products, categories, relations)

    print("Результат:")
    result.show()

    spark.stop()

if __name__ == "__main__":
    quick_test()

Результат:
+------------+-------------+
|product_name|category_name|
+------------+-------------+
|      Laptop|    Computers|
|      Laptop|  Electronics|
|       Phone|  Electronics|
|      Tablet|         NULL|
+------------+-------------+

